In [17]:
import pandas as pd

from tqdm import tqdm

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

In [18]:
df = pd.read_csv('./spain_df.csv')
df.drop(columns=["Unnamed: 0"], inplace=True)
#df = df[df['tier'] == 1]
df

,year,country,league,tier,team,position,squad_depth,avg_age,foreigners,avg_market_value,market_value,has_relegated,has_promoted,has_won_titles
0,2004,ES,LaLiga,1,FC Barcelona,1.0,35,24.9,15,7800000,273030000,False,False,False
1,2004,ES,LaLiga,1,Real Madrid,2.0,34,25.9,10,8930000,303700000,False,False,False
2,2004,ES,LaLiga,1,Villarreal CF,3.0,38,24.9,10,1740000,66250000,False,False,False
3,2004,ES,LaLiga,1,Real Betis Balompié,4.0,32,26.6,8,2660000,85150000,False,False,False
4,2004,ES,LaLiga,1,RCD Espanyol Barcelona,5.0,29,26.1,8,1560000,45150000,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,2023,ES,LaLiga2,2,CD Mirandés,18.0,36,23.7,11,576000,20750000,False,False,False
836,2023,ES,LaLiga2,2,SD Amorebieta,19.0,28,27.1,6,455000,12750000,True,False,False
837,2023,ES,LaLiga2,2,AD Alcorcón,20.0,38,26.4,6,298000,11330000,True,False,False
838,2023,ES,LaLiga2,2,FC Andorra,21.0,32,25.9,6,639000,20450000,True,False,False


In [19]:
# Obtén los años únicos y selecciona el 80% para entrenamiento y 20% para prueba
years = df['year'].unique()
years_train, years_test = train_test_split(years, test_size=0.2, random_state=42)

# Filtra los datos para crear los conjuntos de entrenamiento y prueba
train_data = df[df['year'].isin(years_train)]
test_data = df[df['year'].isin(years_test)]

# Divide las características y la variable objetivo
X_train = train_data.drop(columns=['country', 'league', 'team', 'position', 'has_relegated'])
y_train = train_data['has_relegated']

X_test = test_data.drop(columns=['country', 'league', 'team', 'position', 'has_relegated'])
y_test = test_data['has_relegated']


# Comprueba la distribución de años
print("Años en entrenamiento:", years_train)
print("Años en prueba:", years_test)

Años en entrenamiento: [2012 2009 2015 2007 2022 2020 2017 2006 2013 2023 2008 2016 2011 2014
 2018 2010]
Años en prueba: [2004 2021 2019 2005]


In [69]:
criterios = ['gini', 'entropy', 'log_loss']
profundidades = [3, 5, 10, 15, 25, 50, 100, 500]
estimators = [10, 20, 25, 50, 100]
mejor_clf = {'profundidad':'',
              'criterio':'',
              'estimator': '',
              'pesos_personalizados': '',
              'accuracy':'',
              'precision':'',
              'recall':'',
              'f1':''}
mejor_recall = 0

# Asignamos un peso mayor para la clase 'False' y un peso menor para la clase 'True'
pesos_personalizados = ['balanced_subsample', 'balanced', {0: 1, 1: 5}, {0: 1, 1: 10}]

In [64]:
for criterio in tqdm(criterios):
    for estimator in estimators:
        for profundidad in profundidades:
            for peso in pesos_personalizados:
                # Crear el modelo de RandomForest
                rfc = RandomForestClassifier(n_estimators=estimator, criterion=criterio, max_depth=profundidad, random_state=1337, class_weight=peso)
                
                # Entrenar el modelo
                rfc.fit(X_train, y_train)
                
                # Realizar las predicciones
                predicciones = rfc.predict(X_test)
                
                # Calcular las métricas
                acc = accuracy_score(y_test, predicciones)
                prec = precision_score(y_test, predicciones)
                rec = recall_score(y_test, predicciones)
                f1 = f1_score(y_test, predicciones)
                if prec >= 0.3:
                    if rec >= mejor_recall:
                        mejor_recall = rec
                        mejor_clf['profundidad'] = profundidad
                        mejor_clf['criterio'] = criterio
                        mejor_clf['estimator'] = estimator
                        mejor_clf['pesos_personalizados'] = peso
                        mejor_clf['accuracy'] = acc
                        mejor_clf['precision'] = prec
                        mejor_clf['recall'] = rec
                        mejor_clf['f1'] = f1

100%|██████████| 3/3 [00:30<00:00, 10.01s/it]


In [65]:
mejor_clf

{'profundidad': 5,
 'criterio': 'log_loss',
 'estimator': 25,
 'pesos_personalizados': 'balanced',
 'accuracy': 0.6607142857142857,
 'precision': 0.3253012048192771,
 'recall': 0.9642857142857143,
 'f1': 0.4864864864864865}

In [70]:
for criterio in tqdm(criterios):
    for estimator in estimators:
        for profundidad in profundidades:
            for peso in pesos_personalizados:
                # Crear el modelo de RandomForest
                rfc = RandomForestClassifier(n_estimators=estimator, criterion=criterio, max_depth=profundidad, random_state=1337, class_weight=peso)
                
                # Entrenar el modelo
                rfc.fit(X_train, y_train)
                
                # Realizar las predicciones
                predicciones = rfc.predict(X_test)
                
                # Calcular las métricas
                acc = accuracy_score(y_test, predicciones)
                prec = precision_score(y_test, predicciones)
                rec = recall_score(y_test, predicciones)
                f1 = f1_score(y_test, predicciones)
                if f1 >= mejor_recall:
                    mejor_recall = f1
                    mejor_clf['profundidad'] = profundidad
                    mejor_clf['criterio'] = criterio
                    mejor_clf['estimator'] = estimator
                    mejor_clf['pesos_personalizados'] = peso
                    mejor_clf['accuracy'] = acc
                    mejor_clf['precision'] = prec
                    mejor_clf['recall'] = rec
                    mejor_clf['f1'] = f1

100%|██████████| 3/3 [00:33<00:00, 11.13s/it]


In [71]:
mejor_clf

{'profundidad': 5,
 'criterio': 'log_loss',
 'estimator': 20,
 'pesos_personalizados': 'balanced_subsample',
 'accuracy': 0.6845238095238095,
 'precision': 0.34177215189873417,
 'recall': 0.9642857142857143,
 'f1': 0.5046728971962616}

In [73]:
rfc = RandomForestClassifier(n_estimators=mejor_clf['estimator'], criterion=mejor_clf['criterio'], max_depth=mejor_clf['profundidad'], random_state=1337, class_weight=mejor_clf['pesos_personalizados'])
rfc        

RandomForestClassifier(class_weight='balanced_subsample', criterion='log_loss',
                       max_depth=5, n_estimators=20, random_state=1337)

In [74]:
rfc.fit(X_train, y_train)
predicciones = rfc.predict(X_test)

array([False, False,  True, False,  True, False, False, False,  True,
        True, False, False, False, False, False,  True,  True,  True,
        True,  True, False, False, False, False,  True, False, False,
        True, False,  True, False, False,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False,  True, False, False,  True,
        True, False, False, False, False,  True,  True,  True, False,
       False, False,  True, False,  True,  True,  True,  True, False,
       False,  True, False,  True,  True,  True,  True, False,  True,
        True,  True,  True, False, False, False, False, False, False,
        True, False, False,  True, False, False, False,  True, False,
        True, False,  True,  True, False, False, False, False, False,
        True, False,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False,

In [86]:
nuevo_df = pd.read_csv('./spain_df.csv')
nuevo_df.drop(columns=["Unnamed: 0"], inplace=True)
nuevo_df = nuevo_df[nuevo_df['tier'] == 2]
predict_df = nuevo_df.drop(columns=['country', 'league', 'team', 'position', 'has_relegated'])
predict_df = predict_df[predict_df['year'] == 2023]

predict_df

,year,tier,squad_depth,avg_age,foreigners,avg_market_value,market_value,has_promoted,has_won_titles
818,2023,2,39,25.8,9,741000,28900000,True,False
819,2023,2,45,25.9,19,1130000,50900000,True,False
820,2023,2,35,27.5,9,905000,31680000,False,False
821,2023,2,44,25.0,10,1310000,57430000,True,False
822,2023,2,41,25.3,16,768000,31500000,False,False
823,2023,2,39,25.9,7,929000,36250000,False,False
824,2023,2,36,26.1,6,738000,26550000,False,False
825,2023,2,40,24.3,6,804000,32150000,False,False
826,2023,2,33,26.3,6,622000,20530000,False,False
827,2023,2,29,28.6,2,641000,18600000,False,False


In [87]:
predicciones_nuevo_df = rfc.predict(predict_df)

# Add 'team' and 'position' to nuevo_df (if needed, you can keep them for the final prediction)
predict_df['team'] = nuevo_df['team']
predict_df['position'] = nuevo_df['position']

# Add the predictions to the DataFrame
predict_df['predicciones'] = predicciones_nuevo_df

In [88]:
predict_df

,year,tier,squad_depth,avg_age,foreigners,avg_market_value,market_value,has_promoted,has_won_titles,team,position,predicciones
818,2023,2,39,25.8,9,741000,28900000,True,False,CD Leganés,1.0,False
819,2023,2,45,25.9,19,1130000,50900000,True,False,Real Valladolid CF,2.0,False
820,2023,2,35,27.5,9,905000,31680000,False,False,SD Eibar,3.0,False
821,2023,2,44,25.0,10,1310000,57430000,True,False,RCD Espanyol Barcelona,4.0,False
822,2023,2,41,25.3,16,768000,31500000,False,False,Sporting Gijón,5.0,False
823,2023,2,39,25.9,7,929000,36250000,False,False,Real Oviedo,6.0,False
824,2023,2,36,26.1,6,738000,26550000,False,False,Racing Santander,7.0,False
825,2023,2,40,24.3,6,804000,32150000,False,False,Levante UD,8.0,False
826,2023,2,33,26.3,6,622000,20530000,False,False,Burgos CF,9.0,False
827,2023,2,29,28.6,2,641000,18600000,False,False,Racing Ferrol,10.0,False
